In [56]:
import torch
import numpy as np
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch import nn, tensor, Tensor
from datasets import load_dataset
from transformers import AutoModelForMaskedLM, AutoTokenizer

%matplotlib inline

In [57]:
modelCheckPoint = 'bert-large-cased'
preModel = AutoModelForMaskedLM.from_pretrained(modelCheckPoint)
preTokenizer = AutoTokenizer.from_pretrained(modelCheckPoint)

In [ ]:
def TokenText(tokenizer, text):
    return tokenizer(text, return_tensors = "pt")

def GetResult(
    model,
    tokenizer,
    text,
    options
) -> str:
    """
    Args:
        model:          语言模型
        tokenizer:      编码器
        text:           上下文
        options:        给予的选项
    Returns:
        返回模型从给予的选项中概率最大
    """
    
    text = text.replace('_', '[MASK]')
    # 获得编码
    textToken = TokenText(tokenizer, text)
    textLogits = model(**textToken).logits

    # 获得掩码位置的概率
    maskTokenIndex = torch.where(textToken['input_ids'] == tokenizer.mask_token_id)[1]
    maskLogits = textLogits[0, maskTokenIndex, :][0]

    # 获得选项编码 并且去除头尾的标签
    optionsToken = TokenText(tokenizer, options)['input_ids'][0]
    optionsToken = optionsToken[1 : len(optionsToken) - 1]

    optionsLogits = tensor([maskLogits[index] for index in optionsToken])
    
    # 获得 答案下标
    resIndex = optionsLogits.argmax()

    return tokenizer.decode(optionsToken[resIndex])

'a'

In [ ]:
resIndex = GetResult(preModel, preTokenizer, "This is _ model", "an fantastic Machillka a")

resIndex

'a'